# Compilers and Languages

In [ ]:
!pip install textx
import deborahscript
import utils

## Simple GCD Program

This program computes the GCD of two numbers. Run the cell and hit Enter after typing in each number.

In [ ]:
deborahscript.run("gcd.ds")

## What does this function do?
Run the cell and try entering a few different numbers.

In [ ]:
%%ds
AsSiGN x EqUaLs (INTINPUT) EnDLinE
AsSiGN a EqUaLs (1) EnDLinE
LoOP (x) BeGIn
    a EqUaLs ((a) * (x)) EnDLinE
    x EqUaLs ((x) - (1)) EnDLinE
EnDLinE
PrInT (a) EnDLinE

## Let's write our own language!
Go ahead and open up the file labeled youscript.tx, and fill in the `REPLACE_ME`'s with your own text!

After that, click Kernel -> Restart & Clear Output, re-run the first cell, and jump here to test out your new language!

In [ ]:
%%ys


# Operating Systems

## Threads Demo

This cell creates two Python threads that execute independently. Each thread prints the numbers 0 through 14 in order. You should see the output from both threads interleaving (at least a little bit)!

Once each thread has finished printing, you can stop the cell by pressing the STOP button (looks like a black square). Otherwise, it will keep running forever without producing any more output.

In [ ]:
import _thread
from time import sleep

# Define a function for the thread
def print_arr( threadName, array):
    for el in array:
        print(threadName, el)

# Create two threads as follows
try:
    _thread.start_new_thread( print_arr, ("Thread-1", range(0,15), ) )
    _thread.start_new_thread( print_arr, ("Thread-2", range(0,15), ) )
except:
    print("Error: unable to start thread")
    
while (1):
    pass

Now lets take a look at an example of a race condition, to understand when threading can go wrong!

Let's say you and your friend are on vacation. You both share a checking account. When either of you wakes up, you immediately check the balance, decide to withdraw half for breakfast, brush your teeth, then withdraw the money and leave for breakfast.

As an example, if your account starts at \\$100, and you wake up before your friend, you get \\$50 for breakfast, and they only get \\$25 (the early bird gets the worm!).

Let's define some classes to help with this example. The first is the bank account.

In [ ]:
class BankAccount(object):
    def __init__(self, balance):
        self.balance = balance
    def check_balance(self):
        return self.balance
    def withdraw(self, amt):
        self.balance -= amt
        return amt

Now we define the Person class. Our first implementation will be faulty, so we'll name this class FaultyPerson.

In [ ]:
from threading import Thread

class FaultyPerson(Thread):
    def __init__(self, name, acct):
        super(FaultyPerson, self).__init__()
        self.name = name
        self.acct = acct
        
    # Wakeup operation
    def run(self):
        amt_to_withdraw = self.acct.check_balance()//2
        # Need to brush my teeth!
        sleep(3)
        self.acct.withdraw(amt_to_withdraw)
        print(self.name + ": I got: $" + str(amt_to_withdraw))
        print(self.name + ": Done!")

Now we simulate the above example! When you see done printed twice, stop the cell!

In [ ]:
acct = BankAccount(100)
me = FaultyPerson('me', acct)
friend = FaultyPerson('friend', acct)

# Create two threads as follows
print("Starting Me!")
me.start()
# Our friend's a late riser!
sleep(2)
print("Starting Friend!")
friend.start()

while(1):
    pass

Now let's check our account balance!

In [ ]:
print(acct.check_balance())

Oh no! It seems that we may have to cut our trip short - we were supposed to be left with \\$25, but now we've blown through the whole budget for the trip just on breakfast!

Let's try this again, but this time with locking! Before you check the balance, you're going to get a lock on your account. Then, you'll check the balance, brush your teeth, withdraw your breakfast money, and release the lock on your account. When your friend wakes up, they'll try to get a lock on your account, but won't be allowed to. In effect, you've told your friend, "Hey, I'm in the middle of a *critical* operation involving our bank account - don't touch it till I'm done please!" They'll wait till you're done, and then repeat the same operation as you.

In [ ]:
class Person(Thread):
    def __init__(self, name, acct, acctLock):
        super(Person, self).__init__()
        self.name = name
        self.acct = acct
        self.acctLock = acctLock
        
    # Wakeup operation
    def run(self):
        print(self.name + ": Grabbing lock!")
        self.acctLock.acquire()
        print(self.name + ": Grabbed lock!")
        amt_to_withdraw = self.acct.check_balance()//2
        # Need to brush my teeth!
        sleep(3)
        self.acct.withdraw(amt_to_withdraw)
        print(self.name + ": I got: $" + str(amt_to_withdraw))
        self.acctLock.release()
        print(self.name + ": Done!")

Now we rewind the clock to today morning, and try again! Notice the inclusion of the variable acctLock - this is the lock we are using on your shared account.

In [ ]:
from threading import Lock
acct = BankAccount(100)
acctLock = Lock()
me = Person('me', acct, acctLock)
friend = Person('friend', acct, acctLock)

# Create two threads as follows
print("Starting Me!")
me.start()
# Our friend's a late riser!
sleep(2)
print("Starting Friend!")
friend.start()

while(1):
    pass

We see that you got \\$50, and your friend got \\$25, as expected. We can check our balance, and see that it is the correct amount (\\$25)!

In [ ]:
print(acct.check_balance())

Now you and your friend are free to continue enjoying your vacation!

## Virtual Memory

Try running the small program below called "crash". It tries to write over other apps' memory. Without virtual memory, "crash" might be able to bring down your entire computer, but with virtual memory, its effects will be much more limited. Run it below.

In [ ]:
%%sh
./crash

See how it says "Segmentation fault"? That's a fancy way of saying that the `crash` program just crashed. That's because the operating system detected that it was attempting to access other programs' memory, said, "you've been a bad program", and stopped it. It wasn't able to damage other programs, because it was isolated from other programs' memory.

An aside: You may be wondering what **exactly** this program is doing. Run `!cat crash.c` in the cell below this to find out.

In [ ]:
!cat crash.c

## The Filesystem

### Understanding Directories and the ls command

One function of operating systems is to protect users from snooping on each other's activities. Operating systems implement this by using a permissions system. Let's see that in action.

First, let's look at your own files:

In [ ]:
!ls

The `ls` command lets us take a look at the items in your current directory in the filesystem. You can think of your current directory as the environment in which you are currently operating in. You can see that these are items from the directory of the github binder we have set up for you. What is the name of this directory that we are in? You can find out your current directory with the pwd command (which stands for pwd)

In [ ]:
!pwd

You should see the directory `/home/jovyan/session5`, Don't worry about the specifics of jovyan (this is a jupiter notebook thing)

Looking back at the results of the 'ls' command, some of these are files, like `deborahscript.txt`. The other item type  you see is a directory. An example of this is `filesys`. We can say that `filesys` is a *subdirectory* of the `session5` directory. 

#### Use the ls command followed by the directory you want to look into to see the items in the filesys directory (remember to  use ! before each command)

In [ ]:
!ls filesys

### cat, cp, and rm

Now you can see a list of files in the `filesys` directory. Lets take a look at several command line tools we can use to have a better understanding of the filesystem.

For files that contain text, we are given the built in `cat` command. This command allows us to look at the text inside of a file. The form of the command is `cat <file path>`. 

#### Try to read the contents of `thetext.txt` (remember we can't just say the filename because we are not in the filesys directory). 

For example if we wanted to call a funtion on the file `car.txt`, we would need to call `<function_name> filesys/car.txt`

Once you are able to display the text in `thetext.txt`, try and display the content of `empty.txt` (hint, should be empty)

Now let's say we wanted to make a copy of `thetext.txt`, we can use the `cp` command. The copy command creates a copy of a given file. It has the form `cp <source_file_path> <destination_path>`. Lets say that you wanted to copy the file `empty.txt` to a new file called `stillempty.txt` (both in filesys directory). We would use the form `cp filesys/empty.txt filesys/stillempty.txt`.

#### Now use the command to make a copy of `thetext.txt` into a new file of your choice.

Now that we have the copy of the file, lets say that we wanted to now remove the original. We can use the `rm` command. This command allows us to remove files and directories. For example lets say that we wanted to remove the `emptytext.txt` file, we would call `rm filesys/emptytext.txt`.
#### Use rm to delete the original thetext.txt that we just copied. Then use an ls command to see that the file was deleted (and renamed from the previous section)

Now we just copied a file to a new name and deleted the original. This is similar to simply renaming a file. Luckily we have the `mv` command that does just this. `mv` takes a file and "moves" it to a new location with a specified name (similar to renaming). The format for a `mv` call is: `mv  <source_file_path> <destination_path>`. For example if we wanted to rename `emptytext.txt` to `useless.txt`, we would call `move filesys/emptytext.txt filesys/useless.txt`
#### Use mv to move/rename the file that you created back to thetext.txt (keep it in the same directory). Use an ls command to confirm this was done correctly